# GRPH exercise

Given: A collection of DNA strings in FASTA format having total length at most 10 kbp.

Return: The adjacency list corresponding to O3. You may return edges in any order.

In [ ]:
# module for working with graphs
import networkx as nx

# the first functicon reads a FASTA file
def parser(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    sequences = {}
    label = None
    for line in lines:
        if line.startswith('>'): 
            label = line[1:].strip()
            sequences[label] = ''
        else:
            sequences[label] += line.strip()
    return sequences

# the second function builds a graph with an edge from one sequence to 
# another if the end of the first matches the start of the second
def overlap(sequences, k=3):
    graph = nx.DiGraph()                                          # empty graph
    for label1, seq1 in sequences.items():                    # for every pair of label and sequence 
        for label2, seq2 in sequences.items():
            if label1 != label2 and seq1[-k:] == seq2[:k]:    # if the suffix ofone sequence overlaps the prefix of another 
                graph.add_edge(label1, label2)                    # add an edge to the graph
    return graph

# the third function is the one that actually reads th FASTA file and adds the edges to the graph by recalling the 2 previous functions
def main(input_file):
    sequences = parser(input_file)

    graph = overlap(sequences, k=3)

    for edge in graph.edges():
        print(edge[0], edge[1])

if __name__ == '__main__':
    main('rosalind_grph.txt') 

# TREE exercise 

Given: A positive integer n(n≤1000) and an adjacency list corresponding to a graph on n nodes that contains no cycles.

Return: The minimum number of edges that can be added to the graph to produce a tree.

In [ ]:
import networkx as nx

# the first function reads a graph form a file and returns its objects and n of nodes
def graph_reader(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    n = int(lines[0].strip())                                              # number of nodes in the graph
    edges = [tuple(map(int, line.strip().split())) for line in lines[1:]]  # list of the edges

    graph = nx.Graph()
    graph.add_nodes_from(range(1, n + 1))                                      # adds nodes to the graph
    graph.add_edges_from(edges)                                                # adds edges to the graph
    return graph, n

# the second functions calculate the minimum number of edges required to make the graph a tree
def minimum_edges2tree(graph):
    cc = nx.number_connected_components(graph)                                 # number of connected components
    return cc - 1                                                          # mimimun number of edges to connect all components

# the third function is the one that actually reads the graph and prints the minimum required edges by recalling the 2 previous functions
def main(input_file):
    graph, n = graph_reader(input_file)
    min_edges = minimum_edges2tree(graph)
    print(min_edges)

if __name__ == '__main__':
    main('rosalind_tree.txt')

# LONG exercise 

Given: At most 50 DNA strings of approximately equal length, not exceeding 1 kbp, in FASTA format 
       (which represent reads deriving from the same strand of a single linear chromosome).
       The dataset is guaranteed to satisfy the following condition: 
       there exists a unique way to reconstruct the entire chromosome from these reads by gluing 
       together pairs of reads that overlap by more than half their length.

Return: A shortest superstring containing all the given strings (thus corresponding to a reconstructed chromosome).

In [ ]:
# module from Biopython for parsing FASTA files
from Bio import SeqIO

# the first funciton finds the longest overlap between two strings and generates the combined string
def overlaps(a, b):
    combined = []
    overlapped = []

    for element in range(len(a)):                          # for every element in range a
        if a[element:] == b[:len(a) - element]:            # if the suffix of a overlaps with the prefix of b
            overlapped.append(a[element:])                 # we append the overlapping part to overlapped
            combined.append(a + b[len(a) - element:])      # and merge a with the non-overlapping part of b to form a combined string
            break

    for element in range(len(b)):                          # for every element in range b
        if b[element:] == a[:len(b) - element]:            # if the suffix of b overlaps with the prefix of a 
            overlapped.append(b[element:])                 # we append the overlapping part to overlapped
            combined.append(b + a[len(b) - element:])      # and merge b with the non-overlapping part of a to form a combined string
            break

    if not overlapped:                                     # if no overlap is found
        return '', ''                                      # an empty string is return

    # return the shortest comined string and the longest overlap
    combine_string = min(combined, key=len)
    overlap_string = max(overlapped, key=len)
    return combine_string, overlap_string

# the second function find the shortest superstring 
def superstring(strings):
    temp = strings

    while len(temp) > 1:
        most_overlapping_string = ""
        most_overlapping_string_pair = []
        most_overlapping_string_length = 0

        # with the for loop we compare all pairs of sequences to find the best overlap
        for element in range(len(temp) - 1):
            for element2 in range(element + 1, len(temp)):
                combine_string, overlap_string = overlaps(temp[element], temp[element2])
                if len(overlap_string) > most_overlapping_string_length:
                    most_overlapping_string = combine_string
                    most_overlapping_string_pair = [temp[element], temp[element2]]
                    most_overlapping_string_length = len(overlap_string)

        temp.remove(most_overlapping_string_pair[0])
        temp.remove(most_overlapping_string_pair[1])
        temp.append(most_overlapping_string)
    return temp[0]

# the third function is the one that actually reads the FASTA file and finds the shortest superstring by recalling the 2 previous functions
def main(input_file):
    seq_strings = []
    with open(input_file, 'r') as fa:
        for seq_record in SeqIO.parse(fa, 'fasta'):
            seq_strings.append(str(seq_record.seq))

    shortest_superstring = superstring(seq_strings)
    print(shortest_superstring)

if __name__ == '__main__':
    main('rosalind_long.txt')

# DEG exercise 

Given: A simple graph with n≤10^3 vertices in the edge list format.

Return: An array D[1..n] where D[i] is the degree of vertex i.

In [ ]:
# the first function calculates the degree of each node in the graph
def degrees_calculation(n, edges):
    degrees = [0] * n                                # list of 0s with lenght equal to the n of nodes
    
    for a, b in edges:   
        degrees[a - 1] += 1                          # increase the count of degrees for node a 
        degrees[b - 1] += 1                          # increase the count of degrees for node b
    return degrees

# the second function reads the input and calculates the degrees of each node by reaclling the previous function
def main(input_file):
    with open(input_file, 'r') as f:
        first_line = f.readline().strip()
        n = int(first_line.split()[0])

        edges = []
        for line in f:
            u, v = map(int, line.strip().split()) 
            edges.append((u, v))

    degrees = degrees_calculation(n, edges)
    print(*degrees)

if __name__ == '__main__':
    main('rosalind_deg.txt')

# DDEG exercise 

Given: A simple graph with n≤10^3 vertices in the edge list format.

Return: An array D[1..n] where D[i] is the sum of the degrees of i's neighbors.

In [ ]:
# the first function calculates the degree of each node in the graph 
def degrees_calculation(n, edges):
    degrees = [0] * n                                # list of 0s with lenght equal to the n of nodes
    
    for a, b in edges:   
        degrees[a - 1] += 1                          # increase the count of degrees for node a 
        degrees[b - 1] += 1                          # increase the count of degrees for node b
    return degrees

# the second function calculates the sum of the degrees of each neighbor for each node
def n_degrees_calculation(n, edges, degree):
    adjacency_list = [[] for element in range(n)]    # list to store the neighbors of each node
    
    for a, b in edges:                               # for every a and b in edges
        adjacency_list[a - 1].append(b - 1)          # we append b-1 to the list of a-1
        adjacency_list[b - 1].append(a - 1)          # we append a-1 to the list of b-1
    n_degree_sum = [0] * n                           # list to store the sum of neighb or degrees for each node

    for element in range(n):                         # for every element in range n                      
        sum_degrees = 0
        for neighbor in adjacency_list[element]:
            sum_degrees += degree[neighbor] 
        n_degree_sum[element] = sum_degrees
    return n_degree_sum

# the third function reads the graph data, computes and sums the neighbor degrees for each node by recalling the previuos functions
def main(input_file):
    with open(input_file, 'r') as f:
        first_line = f.readline().strip()
        n = int(first_line.split()[0])
        
        edges = []
        for line in f:
            u, v = map(int, line.strip().split())
            edges.append((u, v))

    degree = degrees_calculation(n, edges)
    result = n_degrees_calculation(n, edges, degree)

    print(*result)

if __name__ == '__main__':
    main('rosalind_ddeg.txt') 

# CC exercise 

Given: A simple graph with n≤10^3 vertices in the edge list format.

Return: The number of connected components in the graph.

In [ ]:
import networkx as nx

# the function reads a graph and computes the number of connected components
def main(input_file):
    with open(input_file, 'r') as file:
        lines = file.readlines()

    num_vertices, num_edges = map(int, lines[0].strip().split())                # extracts the number of vertices and edges

    edge_list = [tuple(map(int, line.strip().split())) for line in lines[1:]]   # reads and converts edge pairs

    graph = nx.Graph()
    graph.add_nodes_from(range(1, num_vertices + 1))                            # add nodes with IDs from 1 to the maximum number of vertices
    graph.add_edges_from(edge_list)                                             # add edges

    print (nx.number_connected_components(graph))                               # calculate number of connected components

if __name__ == '__main__':
    main('rosalind_cc.txt') 

# BFS exercise 

Given: A simple directed graph with n≤10^3 vertices in the edge list format.

Return: An array D[1..n] where D[i] is the length of a shortest path from the vertex 1 to the vertex i (D[1]=0). 
If i is not reachable from 1set D[i] to -1.

In [ ]:
import networkx as nx

# the first function computes the shortest path from one node to all the other nodes in a graph
def shortest_paths(input_file):
    with open(input_file, 'r') as f:
        lines = f.readlines()

    n, e = map(int, lines[0].split())                                   # extract the number of nodes and edges
    edges = [tuple(map(int, line.split())) for line in lines[1:]]       # parse edges from the input file

    graph = nx.DiGraph()
    graph.add_edges_from(edges)                                         # add edges to the graph

    d = [-1] * n                                                        # initialized distance
    d[0] = 0                                                            # distance to itself = 0 
    
    lengths = nx.single_source_shortest_path_length(graph, source=1)    # shortest path lenghts from node 1

    for node, length in lengths.items():
        d[node - 1] = length
    return d

# the second function computes and prints the shortest path by recalling the previous function
def main(input_file):
    result = shortest_paths(input_file)
    print(' '.join(map(str, result)))

if __name__ == '__main__':
    main('rosalind_bfs.txt')

# DAG exercise 

Given: A positive integer k≤20 and k simple directed graphs in the edge list format with at most 10^3 vertices each.

Return: For each graph, output "1" if the graph is acyclic and "-1" otherwise.

In [ ]:
import networkx as nx

# the first function checks if each graph is acyclic
def acyclicity(input_file):
    results = []                                                   # empty list to store the results for each graph
    with open(input_file, 'r') as f:
        lines = [line.strip() for line in f if line.strip()] 
    
    n_graphs = int(lines[0])                                       # number of graphs to check
    index = 1 

    for element in range(n_graphs):                                # for every element within the number og graphs' range        
        n, e = map(int, lines[index].split())                      # calculate the numebr of nodes and edges
        index += 1

        # read the edges
        edges = [tuple(map(int, line.split())) for line in lines[index:index + e]]
        index += e

        graph = nx.DiGraph()                                       # create a graph
        graph.add_edges_from(edges)                                # and add edges to it

        if nx.is_directed_acyclic_graph(graph):                    # if the graph is acyclic
            results.append('1')                                    # append 1 to the empty list result
        else:
            results.append('-1')                                   # if not, append -1 to the empty list result
    return results

# the second function checks acyclicity by recalling the previous function
def main(input_file):
    results = acyclicity(input_file)
    print(' '.join(results))

if __name__ == '__main__':
    main ('rosalind_dag.txt')

# BIP exercise 

Given: A positive integer k≤20 and k simple graphs in the edge list format with at most 10^3 vertices each.

Return: For each graph, output "1" if it is bipartite and "-1" otherwise.

In [ ]:
import networkx as nx

# the first function checks if each graph is bipartite
def bipartite(input_file):
    results = []                                                 # empty list to store the results fro each graph
    with open(input_file, 'r') as f:
        lines = [line.strip() for line in f if line.strip()] 
    
    n_graphs = int(lines[0])                                     # number of graphs to check
    index = 1

    for element in range(n_graphs):                              # for every element within the number og graphs' range        
        n, e = map(int, lines[index].split())                    # calculate the numebr of nodes and edges
        index += 1

        # read the edges
        edges = [tuple(map(int, line.split())) for line in lines[index:index + e]]
        index += e

        graph = nx.Graph()                                       # create a graph
        graph.add_edges_from(edges)                              # and add edges to it

        if nx.is_bipartite(graph):                               # if the graph is bipartite
            results.append('1')                                  # append 1 to the empty list result
        else:
            results.append('-1')                                 # if not, append -1 to the empty list result

    return results

# the second function checks bipartiteness by recalling the previous function
def main(input_file):
    results = bipartite(input_file)
    print(' '.join(results))

if __name__ == '__main__':
    main('rosalind_bip.txt')

# DIJ exercise 

Given: A simple directed graph with positive edge weights from 1 to 10^3 and n≤10^3 vertices in the edge list format.

Return: An array D[1..n] where D[i] is the length of a shortest path from the vertex 1 to the vertex i (D[1]=0). 
If i is not reachable from 1 set D[i] to -1.

In [ ]:
# the first function computes the shortest path from one node to all other nodes in a graph
def shortest_paths(input_file):
    with open(input_file, 'r') as f:
        lines = f.readlines()

    n, e = map(int, lines[0].split())                                          # extract the number of nodes and edges
    edges = [tuple(map(int, line.split())) for line in lines[1:]]              # analyze the edges from the input file

    graph = {i: [] for i in range(1, n + 1)}                                   # adjacency list for the graph

    for a, b, c in edges:                                                      # for every a, b and c
        graph[a].append((b, c))                                                # we append teir edges and weights to the list

    d = {i: float('inf') for i in range(1, n + 1)}                             # we set the distances with infinity 
    d[1] = 0                                                                   # and the starting node with distance 0
    
    pq = [(0, 1)]                                                              # starting node
    
    while pq:                                                    
        current_dist, current_node = pq.pop(0)                                 # extract the node with the smallest distance       
        
        if current_dist > d[current_node]:                                     # check if the distance is no longer the smallest
            continue

        for neighbor, weight in graph[current_node]:                           # for every neighboring node
            distance = current_dist + weight                                   # we compute the new distance
            if distance < d[neighbor]:                                         # if a shorter path is found
                d[neighbor] = distance                                         # we update the distance
                pq.append((distance, neighbor))                                # and add the neighbor to the queue
                pq = sorted(pq)  
                
    result = [d[i] if d[i] != float('inf') else -1 for i in range(1, n + 1)]   # we set -1 for unreachable nodes
    return result

# the second function computes and prints the shortest path by recalling the previous function
def main(input_file):
    result = shortest_paths(input_file)
    print(' '.join(map(str, result)))

if __name__ == '__main__':
    main('rosalind_dij.txt')

# BF exercise 

Given: A simple directed graph with integer edge weights from -10^3 to 10^3 and n≤10^3 vertices in the edge list format.

Return: An array D[1..n] where D[i] is the length of a shortest path from the vertex 1 to the vertex i (D[1]=0). 
If i is not reachable from 1 set D[i] to x.

In [ ]:
# the first function computes the shortest paths from a node to all other nodes in a graph with support for negative weights.
def bf(n, edges, source=1):
    distances = [float('inf')] * (n + 1)                                           # we initialize distances to all nodes as infinity
    distances[source] = 0                                                          # with the distance to the source node as 0

    for element in range(n - 1):             
        for a, b, c in edges:                                                      # for every edge in the graph
            if distances[a] != float('inf') and distances[a] + c < distances[b]:   # if a shorter path is found
                distances[b] = distances[a] + c                                    # we update the distance

    for element in range(1, n + 1):
        if distances[element] == float('inf'):                                     # we set x for unreachable nodes
            distances[element] = 'x'

    return distances[1:] 

# the second function reads the input and computes the shortest paths by recalling the previous function
def main(input_file):
    with open(input_file, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]

    n, e = map(int, lines[0].split())   
    edges = []

    for line in lines[1:]:      
        a, b, c = map(int, line.split())
        edges.append((a, b, c))

    distances = bf(n, edges)       

    print(' '.join(map(str, distances)))  

if __name__ == '__main__':
    main('rosalind_bf.txt')

# CTE exercise

Given: A positive integer k≤20 and k simple directed graphs with positive integer edge weights and at most 10^3 vertices in the edge list format.

Return: For each graph, output the length of a shortest cycle going through the first specified edge if there is a cycle and "-1" otherwise.

In [ ]:
import networkx as nx

# the first function finds the shortest cycle in a graph that includes a given edge
def cycle_edge(edge_list, first_edge):
    graph = nx.DiGraph()                                                        # create a graph
    graph.add_weighted_edges_from(edge_list)                                    # and add weighted edges to it
    
    a, b, c = first_edge                                                        # we extract a (source),b (target) and c (weight of the edge)
    short_cycle = float('inf')                                                  # and set the shortest cycle length as infinity
    found_cycle = False                                                         # found_cycle represent whether or not a cycle has been found

    graph.remove_edge(a, b)                                                     # we remove the edge to check for cycles
    if nx.has_path(graph, b, a):                                                # if there is a path forming a cycle
        len_cycle = nx.shortest_path_length(graph, b, a, weight='weight') + c   # we calculate the cycle length
        short_cycle = min(short_cycle, len_cycle)                               # and update the shortest cycle length
        found_cycle = True                                                      # we mark found_cycle as True
 
    graph.add_edge(a, b, weight=c)                                              # lastly we restore the removed edge
    return short_cycle if found_cycle else -1                                   # and return the shortest cycle length or -1 if no cycle exists

# the second function reads the input and computes the shortest cycle by recalling the previous function
def main(input_file):
    with open(input_file, 'r') as f:
        lines = [line.strip() for line in f if line.strip()] 
    
    n_graphs = int(lines[0])
    idx = 1
    results = []

    for element in range(n_graphs):
        n, e = map(int, lines[idx].split())
        idx += 1

        edge = []
        for element in range(e):
            a, b, c = map(int, lines[idx].split())
            edge.append((a, b, c))
            idx += 1

        edge1 = edge[0]
        result = cycle_edge(edge, edge1)
        results.append(str(result))
    
    print(' '.join(results))

if __name__ == '__main__':
    main('rosalind_cte.txt')

# NWC exercise 

Given: A positive integer k≤20 and k simple directed graphs with integer edge weights from −10^3 to 10^3 and n≤10^3 vertices in the edge list format.

Return: For each graph, output "1" if it contains a negative weight cycle and "-1" otherwise.

In [ ]:
import networkx as nx

# the class helps to manage the directed graph and check for negative cycles
class DirectedGraph:
    def __init__(self):
        self.graph = nx.DiGraph()                                       # we create a graph

    # the first function constructs the graph by adding nodes and edges with weights
    def construct_graph(self, n, edge_list):
        self.graph.add_node(-1)                                         # we add a temporary source node
        for element in range(1, n + 1):                                 # and add nodes from 1 to n
            self.graph.add_node(element)                                # we now connect temporary source to each node with 0 weight
            self.graph.add_edge(-1, element, weight=0)                  # and add edges from the edge list
            
        for a, b, c in edge_list:     
            self.graph.add_edge(a, b, weight=c)
   
    # the second function checks for a negative cycle 
    def has_negative_cycle(self):
        try:
            nx.single_source_bellman_ford_path_length(self.graph, -1)  # we compute distances from the temporary source node
        except nx.NetworkXUnbounded:                                   # if an exception is raised a negative cycle exists
            return '1'                                                 # and we return '1' to indicate a negative cycle
        return '-1'                                                    # if no negative cycle is found we return '-1'

# the third function creates graphs and determines if there is a negative cycle for each graph
def main(input_file):
    with open(input_file, 'r') as f:
        input_lines = f.read().splitlines()
    
    k = int(input_lines[0]) 
    results = []

    i = 1
    while i < len(input_lines):
        n_vertices, n_edges = map(int, input_lines[i].split())
        i += 1

        edge = []
        for element in range(n_edges):
            a, b, c = map(int, input_lines[i].split())
            edge.append((a, b, c))
            i += 1

        graph = DirectedGraph()
        graph.construct_graph(n_vertices, edge)
        results.append(graph.has_negative_cycle())
    
    print(' '.join(results))

if __name__ == '__main__':
    main('rosalind_nwc.txt')

# SDAG exercise 

Given: A weighted DAG with integer edge weights from -10^3 to 10^3 and n≤10^5 vertices in the edge list format.

Return: An array D[1..n] where D[i] is the length of a shortest path from the vertex 1 to the vertex i (D[1]=0). 
If i is not reachable from 1 set D[i] to x.

In [ ]:
import networkx as nx

# the first function calculates the shortest path in a DAG
def sdag(n, edges):
    graph = nx.DiGraph()                                    # create a graph
    graph.add_weighted_edges_from(edges)                    # and add weighted edges to it
    
    order = list(nx.topological_sort(graph))                # we sort the graph
    distances = {i: float('inf') for i in range(1, n + 1)}  # we initialize distances as infinity
    distances[1] = 0                                        # and the distance to the source node as 0

    for a in order:                                         # we now iterate nodes in order
        if distances[a] < float('inf'):                     # if the node is reachable
            for b, attr in graph[a].items():                # we examine the neighbors
                weight = attr['weight']                     # and get the weight of the edge
                if distances[b] > distances[a] + weight:    # if a shorter path is found
                    distances[b] = distances[a] + weight    # we update the distances

    # lastly we replace unreachable nodes with 'x'
    return [distances[i] if distances[i] < float('inf') else 'x' for i in range(1, n + 1)]

# the second function computes the shortest paths in the DAG by recalling the previuos function
def main(input_file):
    with open(input_file, 'r') as f:
        lines = f.read().splitlines()

    n, e = map(int, lines[0].split())

    edges = []
    for line in lines[1:]:
        a, b, c = map(int, line.split())
        edges.append((a, b, c))

    paths = sdag(n, edges)
    
    print(' '.join(map(str, paths)))

if __name__ == '__main__':
    main('rosalind_sdag.txt')